In [1]:
import openai
import pandas as pd

In [2]:
with open('01.txt', 'r') as fh:
    tmp = fh.read()
    itemlist = tmp.split(',')

In [22]:
with open('02.txt', 'r') as fh:
    tmp = fh.read()
    itemlist = tmp.split(',')

In [23]:
itemlist = str(itemlist)

In [24]:
keyfile = open("key.txt", "r")
key = keyfile.readline()
openai.api_key = key

In [25]:
data = [itemlist[0:3200], itemlist[3201:6400], 
        itemlist[6401:9600], itemlist[9601:12800], itemlist[12801:16000]]

In [26]:
data[0]

"['Review \\nBlockchain System in the Higher Education \\nRicardo Raimundo 1 and Albérico Rosário 2', '* \\n\\x01\\x02\\x03\\x01\\x04\\x05\\x06\\x07\\x08\\x01\\x01\\x02\\x03\\x04\\x05\\x06\\x07Citation: Raimundo', ' R.; Rosário', ' A. Blockchain System in the Higher Education. Eur. J. Investig. Health Psychol. Educ. 2021', ' 11', ' 276–293. https://doi.org/10.3390/ ejihpe11010021 \\nAcademic Editors: Maria José Sousa', ' Sónia P. Gonçalves', ' Francesca Dal Mas', ' Davide Calandra and María del Carmen Pérez-Fuentes. \\nReceived: 30 January 2021 Accepted: 14 March 2021 Published: 16 March 2021 \\nPublisher’s Note: MDPI stays neutral with regard to jurisdictional claims in published maps and institutional affl\\xadiations. \\n\\nCopyright: © 2021 by the authors. Licensee MDPI', ' Basel', ' Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). \\n1

In [27]:
def chatgptfn(sub_list):
    result = ''
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant"},
            {"role": "user", "content": f"{sub_list} :give me a summary"}
        ]
    )
    for choice in response.choices:
        result += choice.message.content
    return result

In [28]:
for i in range(0,5):
    data[i] = chatgptfn(data[i])

In [18]:
data[0]

'The paper is a systematic literature review of blockchain technology in education, which includes a bibliometric analysis to gather data on its definitions, methodology, results, and potential uses. The study identifies the benefits, barriers, and applications of the technology in the education sector, as well as potential future areas for its implementation. This is among the first reviews to cover all aspects of blockchain technology in education, making it a valuable resource for educators and researchers.'

In [19]:
data1 = ''
for i in range(0,5):
    data1 = data1 + data[i]

In [20]:
data1

"The paper is a systematic literature review of blockchain technology in education, which includes a bibliometric analysis to gather data on its definitions, methodology, results, and potential uses. The study identifies the benefits, barriers, and applications of the technology in the education sector, as well as potential future areas for its implementation. This is among the first reviews to cover all aspects of blockchain technology in education, making it a valuable resource for educators and researchers.Blockchain technology is a decentralized system that eliminates the need for third-party intermediaries in transactions. It has evolved through four generations, offering features such as smart contracts, securities trading, and governance. It has been applied in various sectors, including finance, government, education, healthcare, energy, public, banking, and business. Blockchain's potential in education is exemplified by platforms like EduCTX, providing a globally trusted credi

In [30]:
data2 = ''
for i in range(0,5):
    data2 = data2 + data[i]

In [31]:
data2

'This article presents a systematic review of research on how blockchain technology is being used in higher education to improve efficiency, effectiveness, privacy control, technological improvement and security of data management mechanisms. The study suggests that blockchain is being used to develop new interventions to improve the ways knowledge data and student records are shared, delivered, and secured. The review integrates 37 articles providing up-to-date knowledge on current implications of blockchain in higher education. The article also suggests some challenges and further research directions.The study looks at how blockchain can contribute to higher education by improving IT infrastructure and computing solutions to promote, maintain, or restore the educational system, and ensuring privacy and security of academic diplomas and degrees. Previous systematic literature reviews (SLRs) have focused on underscoring areas of blockchain application, but a more focused discussion is 

In [ ]:
papers = [data1, data2, ]

In [12]:
import gensim
from gensim import corpora
from pprint import pprint

In [13]:
# 創建詞袋
texts = [[word for word in document.lower().split()] for document in data]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [14]:
# 訓練 LDA 模型
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, 
                                            num_topics=3, random_state=100, update_every=1, 
                                            chunksize=100, passes=10, alpha='auto', per_word_topics=True)

In [15]:
# 輸出主題模型分析結果
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\pecu6\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
